In [1]:
import sys; sys.path.append("../")
import pandas as pd

from lib.match import (
    ColumnsIndex, JaroWinklerSimilarity, StringSimilarity, DateSimilarity, ThresholdMatcher
)
from lib.date import combine_date_columns
from clean.st_tammany_so_pprr import clean
from clean.st_tammany_so_cprr import clean as clean_cprr
from match.st_tammany_so import match_cprr, prepare_post_data, match_cprr_and_post

In [2]:
pprr = clean()
pprr

../lib/clean.py:166: FutureWarning: The default value of regex will change from True to False in a future version.
  return series.str.strip().str.replace(r"[^\w-]+", " ").str.replace(r"\s+", " ")\


,employee_id,last_name,first_name,birth_year,rank_code,rank_desc,pay_group,hire_year,hire_month,hire_day,term_year,term_month,term_day,uid
0,1,divincenti,frank,1923,CG01,school crossing guard,PSAF,1984,09,10,,,,6a22a802ff4f5072bdb2ef289c443e26
1,2,husser,linda,1949,EX04,consultant,AUX,2010,08,03,2016,06,30,af8f11a2cbd55d7e4de29ffaafffbe0d
2,3,jenkins,dickie,1955,MJ3,mj3 corr/comm/crt sec,AUX,1980,11,14,2006,02,03,70e5ff989ccf16676d48d668a08fcbd1
3,4,ranatza,gary,1964,M101,lieutenant protective services,PSAF,1996,09,26,2019,12,12,16e8d29524c6e31728c219dba5945aeb
4,5,kelly,robert,1926,CG01,school crossing guard,PSAF,1994,08,16,2001,10,31,3b029846b7b4037fb1ccf5a41ace73d0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3117,80996,beyer,justin,1974,RESV,reserve deputies,NaN,2020,01,14,,,,85f7e3e5d76557cc71dfbb8b84b56731
3118,80997,barberito,christopher,1990,RESV,reserve deputies,NaN,2020,01,28,,,,892431106b06f986346afabf739f6169
3119,80998,scarlato,michael,1980,RESV,reserve deputies,NaN,2020,02,13,,,,e937a1d07bbe01d1c467a04e6d985cf6
3120,80999,rogers,keith,1962,RESV,reserve deputies,NaN,2020,06,16,,,,84821a28cb1e0cec69382c7ec578f973


In [3]:
post = prepare_post_data()
post

,agency,last_name,first_name,employment_status,level_1_cert_date,last_pc_12_qualification_date,data_production_year,hire_year,hire_month,hire_day,uid
uid,,,,,,,,,,,
00e1eab0baad6550566abd8df81c5c40,st tammany parish so,tullier,paul,full-time,2013-04-23,2020-11-05,2020,2012.0,10.0,23.0,00e1eab0baad6550566abd8df81c5c40
26f859fb158376a3ca411a56d2c05363,st tammany parish so,bennett,georgiana,full-time,NaN,2020-11-05,2020,1992.0,4.0,21.0,26f859fb158376a3ca411a56d2c05363
a3645eb912d70d7395338814a82a77ed,st tammany parish so,pohlmann,dalan,full-time,2017-11-22,2020-11-05,2020,2014.0,11.0,10.0,a3645eb912d70d7395338814a82a77ed
05a5b881f096cd969a099697b95ff3fd,st tammany parish so,sharp,jed,full-time,2004-03-24,2020-11-05,2020,2001.0,6.0,4.0,05a5b881f096cd969a099697b95ff3fd
2d57b0e3135a3b223195c615c52a5fc8,st tammany parish so,fandal,tina,full-time,2010-03-23,2020-11-05,2020,1996.0,1.0,19.0,2d57b0e3135a3b223195c615c52a5fc8
...,...,...,...,...,...,...,...,...,...,...,...
aff4f2a63ba429f9d4afa46440343055,st tammany parish so,morris,phillip,full-time,NaN,NaN,2020,2020.0,4.0,2.0,aff4f2a63ba429f9d4afa46440343055
2bb5f1bdcc198b551423720cf507aafd,st tammany parish so,reuther,robert,full-time,NaN,NaN,2020,2020.0,3.0,11.0,2bb5f1bdcc198b551423720cf507aafd
96c34ff637f32735f979a4724509f388,st tammany parish so,rizan,troy,full-time,NaN,NaN,2020,2020.0,4.0,22.0,96c34ff637f32735f979a4724509f388


In [4]:
cprr = clean_cprr()
cprr

../clean/st_tammany_so_cprr.py:12: FutureWarning: The default value of regex will change from True to False in a future version.
  df.loc[:, col] = df[col].str.replace(r'(\d)\r\n(\d)', r'\1\2')\
../lib/clean.py:166: FutureWarning: The default value of regex will change from True to False in a future version.
  return series.str.strip().str.replace(r"[^\w-]+", " ").str.replace(r"\s+", " ")\
../clean/st_tammany_so_cprr.py:28: FutureWarning: The default value of regex will change from True to False in a future version.
  dept_df.loc[:, 'loc'] = dept_df.loc[:, 'loc'].str.replace(r'\*$', '')
../clean/st_tammany_so_cprr.py:39: FutureWarning: The default value of regex will change from True to False in a future version.
  df.loc[:, 'occur_year'] = df.raw_occur_date.fillna(


,raw_occur_date,department_code,charges,first_name,middle_name,last_name,department_desc,occur_year,occur_month,occur_day,agency,data_production_year,uid,complaint_uid
0,1/16/2015,36,DR.07:0200.400 - Maintenance and care of Vehi...,cody,,talley,Criminal Patrol,2015,1,16,St. Tammany SO,2019,703204a95f12a52f3c782d450a41813d,078bd22ddd94d4b991e6d768800944f4
1,1/18/2015,20,DR.03:09.970.01 (a) - Job Performance - wanton...,shawn,,barrow,St. Tammany Parish Jail,2015,1,18,St. Tammany SO,2019,9e74be39b860aff9b3c14edd764a9ac2,21e15fe83936ad07328d152fad87346d
2,2/7/2015,20,DR.03:09.934.01 - Failure to report - refusal ...,kendall,,twillie,St. Tammany Parish Jail,2015,2,7,St. Tammany SO,2019,00711b033c1b668f21bc65ecc48d8797,392dce325c8cb572f4ec068ea8cd8abe
3,2/13/2015,12,DR.03:09.500.01 - disruptive or abusive behavi...,robert,,jones,Court Security,2015,2,13,St. Tammany SO,2019,d096442b876232add846ec2636c31e98,6c95c979806a914b9a8964e236eebe26
4,2/27/2015,36,SOP.1100:02.000- Property and Evidence Control...,ryan,,suhre,Criminal Patrol,2015,2,27,St. Tammany SO,2019,a0b2227cb977f432c3ca3103e5fdee06,cd88d9b429a9ea9fb488ba534592abba
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
273,"11/5,6,8,13,14,15,26/2019",20,DR.03:09.932.01 Failure to follow supervisor i...,charles,,smith,St. Tammany Parish Jail,2019,NaN,NaN,St. Tammany SO,2019,6313b37cedc742b0ba20f1de5165401e,722e688cc9fcae3abf92fe69b0d0ec44
274,12/19/2019,34,DR.03:09.957.01 Willful disregard of Agency ru...,jonathan,,thornhill,Criminal Patrol,2019,12,19,St. Tammany SO,2019,62aac7a19b72499e45cf88b606fe9820,aa17777916de576025281de3a811ee8e
275,1/1/2020,34,DR. 03:09.977.01 Sleeping on duty,timothy,,scanlan,Criminal Patrol,2020,1,1,St. Tammany SO,2019,1dfc9ef8d18f61fc578df45681503313,fe847e10e93eb99c1c3510c72ba0cfc9
276,Oct-Dec 2019,20,DR.03:09.916.01C Unsatifactory work performance,alexandria,,payne,St. Tammany Parish Jail,2019,NaN,NaN,St. Tammany SO,2019,81e28636c9ea5f88265d8be7faea268b,290e53ef6df3dc61b5c92e9df990784d


In [8]:
cprr[cprr.duplicated(subset=['complaint_uid', 'charges'])]

,raw_occur_date,department_code,charges,first_name,middle_name,last_name,department_desc,occur_year,occur_month,occur_day,agency,data_production_year,uid,complaint_uid
